In [ ]:
# install Kaggel
# !pip install kaggle

# import 
import kaggle
import zipfile
import sqlalchemy as sal
import pandas as pd

# Download Kaggle dataset

# !kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [ ]:
# Specify the path to your zip file
zip_file_path = 'orders.csv.zip'

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()
zip_ref.close()

In [ ]:
# convert not available and unknown data to null
df = pd.read_csv('orders.csv',na_values = ['Not Available', 'unknown'])
df.head(20)
# df['Ship Mode']

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5
5,6,2022-03-13,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,50,50,7,3
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,10,10,4,3
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,860,910,6,5
8,9,2023-03-23,NaN,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,20,20,3,2
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,90,110,5,3


In [15]:
# rename columns because the naming column is not good
# df.rename(columns={'Order Id': 'order_id', 'Order Date':'order_date'})
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3


In [19]:
# add discount column
# df['list_price'] * (df['discount_percent'] / 100)
df['discount'] = df['list_price'] * (df['discount_percent'] / 100)
df.head(5)

# add sales coiumn
# df['list_price'] - df['discount']
df['sale_price'] = df['list_price'] - df['discount']


# add profit column
# df['list_price'] - df['cost_price']
df['profit'] = df['sale_price'] - df['cost_price']

In [ ]:
# date in order_date data type is not a datetime
df.dtypes

# convert order_date object into datetime
df['order_date'] = pd.to_datetime(df['order_date'], format= '%Y-%m-%d')
# df.head(5)
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [23]:
# drop list_price, cost_price, discount_percent because we dont need that we further are analysis that we earlier create three column

df.drop(columns=['list_price', 'cost_price', 'discount_percent'], inplace = True)
# inplace true means it change the actual dataset 

In [25]:
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0


In [ ]:
# connecting engine with computer
connection_string = (
    "mssql://DESKTOP-SU9AB1N/master?"
    "driver=ODBC+Driver+17+for+SQL+Server"
)

engine = sal.create_engine(connection_string)
conn = engine.connect()

In [33]:
# Load data into sql server

# df.to_sql('df_orders',con = conn, index = False, if_exists = 'replace') replace for pandas it will create a datatype big

df.to_sql('df_orders',con = conn, index = False, if_exists = 'append') # we just create table ourself then append data onto it

38